In [ ]:
import pandas as pd

telemetry = pd.read_csv('dataset/PdM_telemetry.csv')
errors = pd.read_csv('dataset/PdM_errors.csv')
maint = pd.read_csv('dataset/PdM_maint.csv')
failures = pd.read_csv('dataset/PdM_failures.csv')
machines = pd.read_csv('dataset/PdM_machines.csv')

# format datetime field which comes in as string
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'], format="%Y-%m-%d %H:%M:%S")

print("Total number of telemetry records: %d" % len(telemetry.index))
print(telemetry.head())
telemetry.describe()

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plot_df = telemetry.loc[(telemetry['machineID'] == 1) & 
                        (telemetry['datetime'] > pd.to_datetime('2015-01-01')) & 
                        (telemetry['datetime'] <pd.to_datetime('2015-02-01')),
                        ['datetime','volt']]
sns.set_style("darkgrid")
plt.figure(figsize=(20, 8))
plt.plot(plot_df['datetime'], plot_df['volt'])
plt.ylabel('voltage')

# make x-axis ticks legible
adf = plt.gca().get_xaxis().get_major_formatter()
adf.scaled[1.0] = '%m-%d-%Y'
plt.xlabel('Date')

# format of datetime field which comes in as string
errors['datetime'] = pd.to_datetime(errors['datetime'],format = '%Y-%m-%d %H:%M:%S')
errors['errorID'] = errors['errorID'].astype('category')
print("Total Number of error records: %d" %len(errors.index))
errors.head()

sns.set_style("darkgrid")
plt.figure(figsize=(20, 8))
errors['errorID'].value_counts().plot(kind='bar')
plt.ylabel('Count')
errors['errorID'].value_counts()

maint['datetime'] = pd.to_datetime(maint['datetime'], format='%Y-%m-%d %H:%M:%S')
maint['comp'] = maint['comp'].astype('category')
print("Total Number of maintenance Records: %d" %len(maint.index))
maint.head()

machines['model'] = machines['model'].astype('category')

print("Total number of machines: %d" % len(machines.index))
machines.head()

# format datetime field which comes in as string
failures['datetime'] = pd.to_datetime(failures['datetime'], format="%Y-%m-%d %H:%M:%S")
failures['failure'] = failures['failure'].astype('category')

print("Total number of failures: %d" % len(failures.index))
failures.head()

import pandas as pd

# Calculate mean values for telemetry features
fields = ['volt', 'rotate', 'pressure', 'vibration']
telemetry_mean_3h = pd.DataFrame()  # Initialize an empty DataFrame

for col in fields:
    # Pivot the telemetry DataFrame and calculate mean values resampled every 3 hours
    temp = pd.pivot_table(telemetry,
                          index='datetime',
                          columns='machineID',
                          values=col).resample('3H', closed='left', label='right').mean().unstack()
    telemetry_mean_3h[col + 'mean_3h'] = temp  # Add mean values to the telemetry_mean_3h DataFrame

telemetry_mean_3h.reset_index(inplace=True)  # Reset index

# Display the resulting DataFrame
telemetry_mean_3h.head()


import pandas as pd

# Calculate mean values for telemetry features
fields = ['volt', 'rotate', 'pressure', 'vibration']
telemetry_mean_3h = pd.DataFrame()  # Initialize an empty DataFrame for mean values
telemetry_sd_3h = pd.DataFrame()    # Initialize an empty DataFrame for standard deviation values

for col in fields:
    # Pivot the telemetry DataFrame and calculate mean values resampled every 3 hours
    temp_mean = pd.pivot_table(telemetry,
                               index='datetime',
                               columns='machineID',
                               values=col).resample('3H', closed='left', label='right').mean().unstack()
    # Add mean values to the telemetry_mean_3h DataFrame
    telemetry_mean_3h[col + 'mean_3h'] = temp_mean
    
    # Pivot the telemetry DataFrame and calculate standard deviation values resampled every 3 hours
    temp_std = pd.pivot_table(telemetry,
                              index='datetime',
                              columns='machineID',
                              values=col).resample('3H', closed='left', label='right').std().unstack()
    # Add standard deviation values to the telemetry_sd_3h DataFrame
    telemetry_sd_3h[col + 'sd_3h'] = temp_std

# Reset index for both DataFrames
telemetry_mean_3h.reset_index(inplace=True)
telemetry_sd_3h.reset_index(inplace=True)

# Display the resulting DataFrame for mean values
telemetry_mean_3h.head()


import pandas as pd

# Calculate rolling mean values for telemetry features
fields = ['volt', 'rotate', 'pressure', 'vibration']
telemetry_mean_24h = pd.DataFrame()  # Initialize an empty DataFrame for rolling mean values
telemetry_sd_24h = pd.DataFrame()    # Initialize an empty DataFrame for rolling standard deviation values

for col in fields:
    # Calculate rolling mean values with a window of 24 hours
    temp_mean = pd.pivot_table(telemetry,
                               index='datetime',
                               columns='machineID',
                               values=col).rolling(window=24).mean().resample('3H', closed='left', label='right').first().unstack()
    # Add rolling mean values to the telemetry_mean_24h DataFrame
    telemetry_mean_24h[col + 'mean_24h'] = temp_mean

    # Calculate rolling standard deviation values with a window of 24 hours
    temp_std = pd.pivot_table(telemetry,
                              index='datetime',
                              columns='machineID',
                              values=col).rolling(window=24).std().resample('3H', closed='left', label='right').first().unstack()
    # Add rolling standard deviation values to the telemetry_sd_24h DataFrame
    telemetry_sd_24h[col + 'sd_24h'] = temp_std

# Drop rows with NaN values for 'voltmean_24h' column
telemetry_mean_24h = telemetry_mean_24h.dropna(subset=['voltmean_24h'])

# Drop rows with NaN values for 'voltsd_24h' column
telemetry_sd_24h = telemetry_sd_24h.dropna(subset=['voltsd_24h'])

# Reset index for both DataFrames
telemetry_mean_24h.reset_index(inplace=True)
telemetry_sd_24h.reset_index(inplace=True)

# Display the resulting DataFrame for rolling mean values
telemetry_mean_24h.head()


import pandas as pd

# Concatenate and merge columns from feature sets
telemetry_feat = pd.concat([telemetry_mean_3h,
                            telemetry_sd_3h.iloc[:, 2:6],
                            telemetry_mean_24h.iloc[:, 2:6],
                            telemetry_sd_24h.iloc[:, 2:6]], axis=1).dropna()

# Display summary statistics of the merged DataFrame
telemetry_feat.describe()


errors

# create a column for each error type
error_count = pd.get_dummies(errors.set_index('datetime')).reset_index()
error_count
error_count.columns = ['datetime', 'machineID', 'error1', 'error2', 'error3', 'error4', 'error5']
error_count.head(13)

# create a column for each error type
error_count = pd.get_dummies(errors.set_index('datetime')).reset_index()
error_count
error_count.columns = ['datetime', 'machineID', 'error1', 'error2', 'error3', 'error4', 'error5']
error_count.head(13)

# combine errors for a given machine in a given hour
error_count = error_count.groupby(['machineID','datetime']).sum().reset_index()
error_count.head(13)

error_count = telemetry[['datetime', 'machineID']].merge(error_count, on=['machineID', 'datetime'], how='left').fillna(0.0)
error_count.describe()

import pandas as pd

# Define fields
fields = ['error%d' % i for i in range(1, 6)]

# Initialize a list to store temporary results
temp = []

# Iterate over each error column
for col in fields:
    # Calculate rolling sum with a window of 24 hours, then resample to 3-hour intervals
    temp.append(pd.pivot_table(error_count,
                               index='datetime',
                               columns='machineID',
                               values=col)
                .rolling(window=24)
                .sum()
                .resample('3H', closed='left', label='right')
                .first()
                .unstack())

# Concatenate the temporary results along the column axis
error_count = pd.concat(temp, axis=1)

# Rename columns
error_count.columns = [i + 'count' for i in fields]

# Reset index
error_count.reset_index(inplace=True)

# Drop rows with NaN values
error_count = error_count.dropna()

# Display summary statistics
error_count.describe()


import numpy as np

# create a column for each error type
comp_rep = pd.get_dummies(maint.set_index('datetime')).reset_index()
comp_rep.columns = ['datetime', 'machineID', 'comp1', 'comp2', 'comp3', 'comp4']

# combine repairs for a given machine in a given hour
comp_rep = comp_rep.groupby(['machineID', 'datetime']).sum().reset_index()

# add timepoints where no components were replaced
comp_rep = telemetry[['datetime', 'machineID']].merge(comp_rep,
                                                      on=['datetime', 'machineID'],
                                                      how='outer').fillna(0).sort_values(by=['machineID', 'datetime'])

components = ['comp1', 'comp2', 'comp3', 'comp4']
for comp in components:
    # convert indicator to most recent date of component change
    comp_rep.loc[comp_rep[comp] < 1, comp] = None
    comp_rep.loc[-comp_rep[comp].isnull(), comp] = comp_rep.loc[-comp_rep[comp].isnull(), 'datetime']
    
    # forward-fill the most-recent date of component change
    comp_rep[comp] = comp_rep[comp].fillna(method='ffill')

# remove dates in 2014 (may have NaN or future component change dates)    
comp_rep = comp_rep.loc[comp_rep['datetime'] > pd.to_datetime('2015-01-01')]

# replace dates of most recent component change with days since most recent component change
for comp in components:
    comp_rep[comp] = (comp_rep['datetime'] - comp_rep[comp]) / np.timedelta64(1, 'D')
    
comp_rep.describe()

telemetry_feat

final_feat = telemetry_feat.merge(error_count, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(comp_rep, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(machines, on=['machineID'], how='left')

print(final_feat.head())
final_feat.describe()

from sklearn.cluster import KMeans
import pandas as pd

# Assuming final_feat is prepared with all relevant features without labels

# Preprocessing: handle missing values
final_feat.fillna(method='bfill', inplace=True)  # Fill missing values using backward fill method
final_feat.fillna(method='ffill', inplace=True)  # Fill remaining missing values using forward fill method

# Drop non-numeric columns
final_feat_numeric = final_feat.select_dtypes(include='number')

# Normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
final_feat_normalized = scaler.fit_transform(final_feat_numeric)

# Determine the optimal number of clusters using the Elbow method
import matplotlib.pyplot as plt
wcss = []  # Within-cluster sum of squares
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(final_feat_normalized)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# From the Elbow Method plot, choose the optimal number of clusters

# Apply KMeans clustering
num_clusters = 3  # Choose the optimal number of clusters based on the Elbow Method
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', random_state=42)
final_feat['cluster'] = kmeans.fit_predict(final_feat_normalized)

# Analyze clusters to identify patterns indicative of potential failure
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=final_feat_numeric.columns)
print(cluster_centers)

# You can further analyze the clusters to identify patterns that might indicate potential failures


from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

# Davies-Bouldin index
db_index = davies_bouldin_score(final_feat_normalized, final_feat['cluster'])
print("Davies-Bouldin Index:", db_index)

# Calinski-Harabasz index
ch_index = calinski_harabasz_score(final_feat_normalized, final_feat['cluster'])
print("Calinski-Harabasz Index:", ch_index)


import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
final_feat_pca = pca.fit_transform(final_feat_normalized)

# Plot the clusters
plt.figure(figsize=(10, 6))
plt.scatter(final_feat_pca[:, 0], final_feat_pca[:, 1], c=final_feat['cluster'], cmap='viridis', alpha=0.5)
plt.title('Clusters Visualization (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Cluster')
plt.grid(True)
plt.show()


# Transform the cluster centers to the PCA space
cluster_centers_pca = pca.transform(kmeans.cluster_centers_)

# Plot the clusters
plt.figure(figsize=(10, 6))
plt.scatter(final_feat_pca[:, 0], final_feat_pca[:, 1], c=final_feat['cluster'], cmap='viridis', alpha=0.5)
plt.scatter(cluster_centers_pca[:, 0], cluster_centers_pca[:, 1], marker='x', s=100, c='red', label='Cluster Centers')
plt.title('Clusters with Cluster Centers (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Cluster')
plt.legend()
plt.grid(True)
plt.show()


# Count the number of data points in each cluster
cluster_counts = final_feat['cluster'].value_counts().sort_index()

# Plot cluster size distribution
plt.figure(figsize=(8, 5))
cluster_counts.plot(kind='bar', color='skyblue')
plt.title('Cluster Size Distribution')
plt.xlabel('Cluster')
plt.ylabel('Number of Data Points')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


import seaborn as sns
import matplotlib.pyplot as plt

# Display the columns to identify suitable features
print(final_feat.columns)

# Select a subset of real feature names for pairwise plots
selected_features = ['voltmean_3h', 'rotatemean_3h', 'pressuremean_3h']  # Replace with actual feature names

# Pairwise plot
sns.pairplot(final_feat[selected_features + ['cluster']], hue='cluster', palette='viridis')
plt.suptitle('Pairwise Feature Plots', y=1.02)
plt.show()



from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming final_feat is already prepared with all relevant features without labels

# Preprocessing: handle missing values
final_feat.fillna(method='bfill', inplace=True)  # Fill missing values using backward fill method
final_feat.fillna(method='ffill', inplace=True)  # Fill remaining missing values using forward fill method

# Drop non-numeric columns
final_feat_numeric = final_feat.select_dtypes(include='number')

# Normalize the data
scaler = StandardScaler()
final_feat_normalized = scaler.fit_transform(final_feat_numeric)

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.3, min_samples=10)  # Adjusted parameters
final_feat['cluster'] = dbscan.fit_predict(final_feat_normalized)

# Number of clusters (excluding noise)
num_clusters = len(set(final_feat['cluster'])) - (1 if -1 in final_feat['cluster'] else 0)
print("Number of clusters:", num_clusters)

# Number of noise points
num_noise = list(final_feat['cluster']).count(-1)
print("Number of noise points:", num_noise)

# Visualize clusters using PCA
from sklearn.decomposition import PCA

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
final_feat_pca = pca.fit_transform(final_feat_normalized)

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=final_feat_pca[:, 0], y=final_feat_pca[:, 1], hue=final_feat['cluster'], palette='viridis', alpha=0.5)
plt.title('DBSCAN Clusters Visualization (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()


from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming final_feat is already prepared with all relevant features without labels

# Preprocessing: handle missing values
final_feat.fillna(method='bfill', inplace=True)  # Fill missing values using backward fill method
final_feat.fillna(method='ffill', inplace=True)  # Fill remaining missing values using forward fill method

# Drop non-numeric columns
final_feat_numeric = final_feat.select_dtypes(include='number')

# Normalize the data
scaler = StandardScaler()
final_feat_normalized = scaler.fit_transform(final_feat_numeric)

# Apply Gaussian Mixture Model
gmm = GaussianMixture(n_components=3, random_state=42)  # You can choose the number of components
final_feat['cluster'] = gmm.fit_predict(final_feat_normalized)

# Visualize clusters using PCA
from sklearn.decomposition import PCA

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
final_feat_pca = pca.fit_transform(final_feat_normalized)

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=final_feat_pca[:, 0], y=final_feat_pca[:, 1], hue=final_feat['cluster'], palette='viridis', alpha=0.5)
plt.title('Gaussian Mixture Model Clusters Visualization (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()




# Assuming final_feat is already prepared with all relevant features without labels

# Preprocessing: handle missing values
final_feat.fillna(method='bfill', inplace=True)  # Fill missing values using backward fill method
final_feat.fillna(method='ffill', inplace=True)  # Fill remaining missing values using forward fill method

# Drop non-numeric columns
final_feat_numeric = final_feat.select_dtypes(include='number')


# Normalize the data
scaler = StandardScaler()
final_feat_normalized = scaler.fit_transform(final_feat_numeric)


# Apply Gaussian Mixture Model
gmm = GaussianMixture(n_components=3, random_state=42)  # You can choose the number of components
final_feat['cluster'] = gmm.fit_predict(final_feat_normalized)


# Visualize clusters using PCA
from sklearn.decomposition import PCA

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
final_feat_pca = pca.fit_transform(final_feat_normalized)


# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=final_feat_pca[:, 0], y=final_feat_pca[:, 1], hue=final_feat['cluster'], palette='viridis', alpha=0.5)
plt.title('Gaussian Mixture Model Clusters Visualization (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()
